In [1]:
import sys
sys.path.insert(1, '../../scripts/')
from s3_support import *

import pandas as pd

# load data

## traffic

In [6]:
q = '''select
            form,
            date_trunc('month', date) as month,
            sum(views) as views
        from ga
        where
            date>='2024-01-01' and
            devicecategory='mobile' and
            path ilike '%sms%'
        group by form, date_trunc('month', date)'''
traff = redshift_query_read(q, schema='production')

In [18]:
print("{:,} entries; {:,} forms".format(len(traff), len(traff['form'].unique())))
traff.tail(2)

65 entries; 38 forms


,form,month,views
63,1012787,2024-07-01,2
64,1024702,2024-03-01,4


## transactions

In [13]:
q = '''select
            date_trunc('month', date) as month,
            form,
            count(id) as transactions,
            sum(amount) as volume
        from transactions
        where
            status='A' and
            source='sms' and
            year>=2024 and
            (recurring=0 or recurring_origin=1)
        group by form, date_trunc('month', date)'''
trans = redshift_query_read(q, schema='production')

In [19]:
print("{:,} entries; {:,} forms".format(len(trans), len(trans['form'].unique())))
trans.tail()

669 entries; 418 forms


,month,form,transactions,volume
664,2024-09-01,1030533,13,712.05
665,2024-09-01,1036774,2,10.40
666,2024-09-01,1019607,1,20.79
667,2024-09-01,1043650,1,25.00
668,2024-09-01,1037575,2,25.30


## merge data

In [20]:
df = trans.merge(traff, on=['month', 'form'], how='left')

In [21]:
df['completion'] = df['transactions'] / df['views']
df.tail()

,month,form,transactions,volume,views,completion
664,2024-09-01,1030533,13,712.05,NaN,NaN
665,2024-09-01,1036774,2,10.40,NaN,NaN
666,2024-09-01,1019607,1,20.79,NaN,NaN
667,2024-09-01,1043650,1,25.00,NaN,NaN
668,2024-09-01,1037575,2,25.30,NaN,NaN


In [23]:
len(df), len(df.dropna()), len(df[df['completion']<1.])

(669, 17, 7)